In [1]:
import pandas as pd  

In [3]:
try:
    df = pd.read_csv('data.csv', sep=';', encoding='latin-1')
except Exception as e:
    print(f"Erreur lors du chargement du fichier: {e}")
    # Si le chargement échoue, nous arrêtons ici.
    raise

In [19]:
df = df[df['Sexe'] != 'Sexe - total']

In [20]:
all_Regions = df['Grande région'].unique()
allowed_regions = [region for region in all_Regions if region != 'Suisse']
print(f"Régions identifiées (excluant 'Suisse'): {len(allowed_regions)} - {allowed_regions}")

Régions identifiées (excluant 'Suisse'): 7 - ['Région lémanique', 'Espace Mittelland', 'Nordwestschweiz', 'Zürich', 'Ostschweiz', 'Zentralschweiz', 'Ticino']


In [21]:
df_for_each_region = {}
nombre_df = 0

for region in allowed_regions:
    # Filtrer le DataFrame principal pour la région actuelle et créer une copie propre (.copy())
    df_for_each_region[region] = df[df['Grande région'] == region].copy()
    nombre_df += 1

print(f"\n✅ Opération terminée. {nombre_df} DataFrames créés avec succès.")


✅ Opération terminée. 7 DataFrames créés avec succès.


In [30]:
print("\n--- Aperçu des DataFrames créés ---")
for region, sub_df in df_for_each_region.items():
    # Afficher la taille de chaque nouveau DataFrame
    print(f"**{region}** : {sub_df.shape[0]} lignes, {sub_df.shape[1]} colonnes")
print("\n\n--- Exemples de données dans chaque DataFrame ---\n")
for region, sub_df in df_for_each_region.items():
    print(f"**{region}** : {sub_df.tail(2).to_markdown(index=False)}\n")
    
    # Afficher les 2 premières lignes du premier DataFrame pour démonstration
    # if region == allowed_regions[0]:
    #     print("Voici les 2 premières lignes du DataFrame pour la première région (Zurich) :")
    #     print(sub_df.head(2).to_markdown(index=False))
    #     print("...")


--- Aperçu des DataFrames créés ---
**Région lémanique** : 180 lignes, 10 colonnes
**Espace Mittelland** : 180 lignes, 10 colonnes
**Nordwestschweiz** : 180 lignes, 10 colonnes
**Zürich** : 180 lignes, 10 colonnes
**Ostschweiz** : 180 lignes, 10 colonnes
**Zentralschweiz** : 180 lignes, 10 colonnes
**Ticino** : 180 lignes, 10 colonnes


--- Exemples de données dans chaque DataFrame ---

**Région lémanique** : |   Année | Grande région    | Division économique         | Position professionnelle   | Sexe   |   Médiane |   P10 |   P25 |   P75 |   P90 |
|--------:|:-----------------|:----------------------------|:---------------------------|:-------|----------:|------:|------:|------:|------:|
|    2012 | Région lémanique | > 63 Services d'information | Sans fonction de cadre     | Femmes |       nan |   nan |   nan |   nan |   nan |
|    2012 | Région lémanique | > 63 Services d'information | Sans fonction de cadre     | Hommes |       nan |   nan |   nan |   nan |   nan |

**Espace Mitt

In [32]:
import numpy as np

df_pay_gap_by_region = {}

cols_to_keep = [ 'Année', 'Grande région', 'Division économique',  'Position professionnelle']

for region, df_region in df_for_each_region.items():
   df_cleaned = df_region.replace(['..', ':', ';;', ''], np.nan)
   df_cleaned['Médiane'] = pd.to_numeric(df_cleaned['Médiane'], errors='coerce')
   df_pivot = df_cleaned.pivot_table(
       index=cols_to_keep,
       columns='Sexe',
       values='Médiane',
      aggfunc='first'
   ).reset_index()
   df_pivot.rename(columns={'Hommes': 'Mediane_Homme', 'Femmes': 'Mediane_Femme'}, inplace=True)
   df_pivot['Pay_Gap'] = df_pivot['Mediane_Homme'] - df_pivot['Mediane_Femme']
   final_cols = cols_to_keep + ['Mediane_Homme', 'Mediane_Femme', 'Pay_Gap']
   df_final = df_pivot[final_cols].copy()
   df_pay_gap_by_region[region] = df_final
   print(f"✅ Calcul du Pay Gap pour la région {region} terminé. Taille du DataFrame: {df_final.shape}")

✅ Calcul du Pay Gap pour la région Région lémanique terminé. Taille du DataFrame: (63, 7)
✅ Calcul du Pay Gap pour la région Espace Mittelland terminé. Taille du DataFrame: (72, 7)
✅ Calcul du Pay Gap pour la région Nordwestschweiz terminé. Taille du DataFrame: (60, 7)
✅ Calcul du Pay Gap pour la région Zürich terminé. Taille du DataFrame: (90, 7)
✅ Calcul du Pay Gap pour la région Ostschweiz terminé. Taille du DataFrame: (67, 7)
✅ Calcul du Pay Gap pour la région Zentralschweiz terminé. Taille du DataFrame: (69, 7)
✅ Calcul du Pay Gap pour la région Ticino terminé. Taille du DataFrame: (55, 7)


In [35]:
print("\n--- Aperçu du Résultat (Région de Zurich) ---")
df_zurich_diff = df_pay_gap_by_region['Zürich']
print(df_zurich_diff.tail().to_markdown(index=False))


--- Aperçu du Résultat (Région de Zurich) ---
|   Année | Grande région   | Division économique         | Position professionnelle               |   Mediane_Homme |   Mediane_Femme |   Pay_Gap |
|--------:|:----------------|:----------------------------|:---------------------------------------|----------------:|----------------:|----------:|
|    2022 | Zürich          | > 63 Services d'information | Cadre inférieur                        |           13255 |           12912 |       343 |
|    2022 | Zürich          | > 63 Services d'information | Cadre supérieur et moyen               |           18057 |           18545 |      -488 |
|    2022 | Zürich          | > 63 Services d'information | Position professionnelle - total       |           15374 |           11421 |      3953 |
|    2022 | Zürich          | > 63 Services d'information | Responsable de l'exécution des travaux |           11196 |             nan |       nan |
|    2022 | Zürich          | > 63 Services d'information |